In [1]:
import network

In [6]:
example = network.Network.from_atoms("/home/sergey/work/auxetic_optimizer/dense_network/coord.dat.1")
example.write_to_file("/home/sergey/work/auxetic_optimizer/dense_network/network.lmp")
example.coordination_number

5.79

In [9]:
example = network.Network.from_data_file("/home/sergey/work/auxetic_optimizer/dense_network/dense_network.lmp", include_angles=False, include_dihedrals=False)

avg = sum([bond.length for bond in example.bonds])/len(example.bonds)
print(avg)
print(example.box.x)
print(example.box.y)

1.2098584153713294
22.153978
22.153978
